In [2]:
import random
import time

#Global parameters
d = 8
e = 5 #w size c
f = 5 #z size \epsilon
n = d+1+e+f
msg_cnt = (3 * f) + 1 #Number of messages
b = 2^60  #Secrets/nonces are sampled with their numerators and denominators in {-(b-1),...,0,...,b-1}
iteration = 1 #This variable is used to iterate through the test()
debug = False


def keygen():
    # M is the secret matrix that is multiplied with the encoded data vector; it corresponds to inverse of M_base matrix in the ICISS paper.
    M = random_matrix(QQ, n, n, num_bound=b, den_bound=b)
    while (not M.is_invertible()):
        M = random_matrix(QQ, n, n, num_bound=b, den_bound=b)

    #Long term secret vector S with (d+1) length
    V = VectorSpace(QQ, (d+1))
    coordinates = [random.randint(-b, b) for _ in range(d+1)]

    # Create the random vector
    s = V(coordinates)

    #A fixed vector W with (c) length
    V1 = VectorSpace(QQ, e)
    coordinates1 = [random.randint(-b, b) for _ in range(e)]

    # Create the random vector
    w = V1(coordinates1)
    return (M, s, w)  # Return the values of M, s, and w

def encrypt(m, s, w, M): #Input m is Message to be encrypted

    if debug:
        print('\n Message vector m:', m)
    #Ephemeral secret z of size \epsilon
    V = VectorSpace(QQ, f)
    coordinates = [random.randint(-b, b) for _ in range(f)]

    # Create the random vector
    z = V(coordinates)  # Create a copy of z
    if debug:
        print('\n Ephemeral vector z:', z)

    #Place holder for pre-processed data
    pre_data = vector(QQ, n)  # Change to a mutable vector
    if debug:
        print('\n Zero Pre-data place holder', pre_data)
    #Fill up w
    for i in range(d+1, d+1+e):
        pre_data[i] = w[i - (d+1)]
    #Fill up z
    for i in range(d+1+e, d+1+e+f):
        pre_data[i] = z[i - (d+1+e)]


    if debug:
        print('\n Pre-data place holder after w and z populated', pre_data)

    #Pre-process the data
    for i in range(0, d):
        pre_data[i] = RR((s[i] - 2 * m[i]))

    #Process d+1 place holder, this is same as norm
    temp = vector(QQ, d)
    for i in range(0, d):
         temp[i] = pre_data[i]
    norm = RR(temp.norm())

    #update the norm at d+1
    pre_data[d] = norm
    if debug:
        print('\n Pre-data place holder after complete pre-processing', pre_data)

    #Multiply pre_data with M
    c = pre_data * M

    return c

def attack(C): #The adversary gets the encrypted data (this could be encryption of A or B)
    not_in_span_cnt = 0
    in_span_cnt = 0

    #C contains encryption of n messages, break it down to c_i
    c = [None] * n # Create a list of length n to store c_i
    for i in range(0, msg_cnt):
        c[i] = C[i]
    if debug:
        print('\nIndividual encrypted data first:', c[0])
    if debug:
        print('\nIndividual encrypted data last:', c[n-1])

    #Difference vector
    D = [None] * (msg_cnt - 1)
    for i in range(0, msg_cnt - 1):
        D[i] = c[i+1] - c[i]
        if debug:
            print('\nDiff vector:', D[i])

    #Define B = (D1, · · · , Dε)
    B = [None] * f
    for i in range(0, f):
        B[i] = D[i]

    B_span = span(B)
    if debug:
        print('\nB Span:', B_span)
    for i in range(f, (3*f)):
        if debug:
            print('\nChecking Span for index', i)
        # Check if the vector is in the span of the vectors
        if debug:
            print('Checking D in B_span', D[i])
        if D[i] in B_span:
            in_span_cnt +=1
            if debug:
                print("The vector is in the span of the vectors.")
        else:
            not_in_span_cnt +=1
            if debug:
                print("The vector is not in the span of the vectors.")

    if debug:
        print('in span cnt and not in span cnt', in_span_cnt, not_in_span_cnt )

    if in_span_cnt > f:
        return 0
    else:
        return 1

# Test the function
def test(toss):
    print('Calling keygen\n')
    M, s, w = keygen()  # Assign the values returned by the function to M, s, and w
    if debug:
        print('\n New Matrix:\n', M)
    if debug:
        print('\nSecret Vector s:\n', s)
    if debug:
        print('\nFixed Vector w:\n', w)

    #Message to be encrypted is a d dimension vector
    #Below code is for a random messsage
    mrand = [[0] * d for _ in range(msg_cnt)]

    for i in range (0, msg_cnt):
        mv = VectorSpace(QQ, d)
        mv_coordinates = [random.randint(-b, b) for _ in range(d)]
        temp = mv(mv_coordinates)
        if debug:
            print(temp)
        for j in range (0, d):
            mrand[i][j] = temp[j]

    if debug:
        print('\n mrand Array:\n', mrand)


    #Below code is used if the message is all zero
    mzero = [[0] * d for _ in range(msg_cnt)]

    for i in range (0, msg_cnt):
        for j in range (0, d):
            mzero[i][j] = 0

    if debug:
        print('\n mzero Array:\n', mzero)

    if debug:
        print('\n Zero Message to be encrypted:', mzero)
    if debug:
        print('\n Non Zero Message to be encrypted:', mrand)
    print('Calling Encrypt Routine\n')

    #Set C (c_1, c_2, .. c_n) is the encryption of (m_1, m_2, ..., m_n)
    C = []  # Define c as a list
    for i in range(0, msg_cnt): #loop for number of messages
        if toss == 0:
            C.append(encrypt(mzero[i], s, w, M))
        else:
            C.append(encrypt(mrand[i], s, w, M))      

    if debug:
        print('Encrypted data:', C)

    print('Calling attack function\n')
    start_time = time.time()
    result = attack(C)
    end_time = time.time()
    execution_time_micro = (end_time - start_time) * 1000000

    print(f"Execution time: {execution_time_micro} microseconds")
    return result
    

def test_correctness():
    correct_guess_cnt = 0
    toss_A_cnt = 0
    toss_B_cnt = 0
    for i in range (0, iteration):
        toss = random.randint(0, 1)
        if toss == 0:
            toss_A_cnt = toss_A_cnt + 1
        else:
            toss_B_cnt = toss_B_cnt + 1
            
        #Uniformly randomly choose between (0,1), If 0 is chosen mzero is used else mrand
        print('\n Iteration:', i, 'Toss:', toss)
        result = test(toss)
        if result == 0:
            print('\nAttack result as encryption of A')
        else:
            print('\nAttack result as encryption of B')
        if toss == result:
            correct_guess_cnt = correct_guess_cnt + 1
    fail_percentage = (iteration - correct_guess_cnt)/(iteration) * 100
    print('Result of test_correctness: Total iterations:', iteration, ' Message A count:', toss_A_cnt, ' Message B count:', toss_B_cnt)
    print('Correct guess:', correct_guess_cnt, 'Success rate:', (100 - fail_percentage), '%\n')
    
    
    
#invoke test function
test_correctness()


 Iteration: 0 Toss: 0
Calling keygen

Calling Encrypt Routine

Calling attack function

Execution time: 59517.38357543945 microseconds

Attack result as encryption of A
Result of test_correctness: Total iterations: 1  Message A count: 1  Message B count: 0
Correct guess: 1 Success rate: 100 %

